In [276]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime,timedelta
import pandas as pd
import numpy as np

In [277]:
bugün=datetime.now().strftime("%Y-%m-%d")
bugün

'2024-10-09'

In [278]:
dün=(datetime.now()-timedelta(days=1)).strftime("%Y-%m-%d")

In [279]:
data=pd.read_csv("migrossepet.csv")
data=data.set_index(data["Unnamed: 0"]).drop("Unnamed: 0",axis=1)
data.index.name=""
data=data.drop_duplicates()
try:
    data=data.drop(f"{bugün}",axis=1)
except:
    pass
data

,Ürün,2024-10-06,2024-10-07,2024-10-08
,,,,
"Alkolsüz içecekler (meşrubat, ayran vb.)",İçim Şişe Ayran 15 Lt,45.95,45.95,45.95
"Alkolsüz içecekler (meşrubat, ayran vb.)",İçim Şişe Ayran 1 L,30.95,30.95,30.95
"Alkolsüz içecekler (meşrubat, ayran vb.)",İçim Çilekli Kefir 1 L,51.5,51.5,51.5
"Alkolsüz içecekler (meşrubat, ayran vb.)",İçim Yeşil Çaylı ve Elmalı Kefir 250 ml,21.95,21.95,21.95
"Alkolsüz içecekler (meşrubat, ayran vb.)",İçim Rahat Laktozsuz Kefir 1 L,53.5,53.5,53.5
...,...,...,...,...
Şeker,Irmak Küp Şeker 1 Kg,41.95,41.95,41.95
Şeker,Migros Toz Şeker 5 Kg,164.5,164.5,164.5
Şeker,Migros Toz Şeker 3 Kg,104.25,104.25,104.25


In [280]:
def veriekle(urun, data, urunler_df):
    # Verileri merge yapıyoruz
    merged_df = pd.merge(data.loc[urun], urunler_df, on='Ürün', how='outer')
    
    # Index'i doğru ürün ismiyle dolduruyoruz
    merged_df.index = len(merged_df) * [urun]

    # Eğer _x ve _y ile aynı tarihli sütunlar varsa birleştiriyoruz
    tarih_sutunlari = [col for col in merged_df.columns if col.endswith("_x") or col.endswith("_y")]
    
    for col in set([col.split("_")[0] for col in tarih_sutunlari]):
        if col + "_x" in merged_df.columns and col + "_y" in merged_df.columns:
            # Sütunları birleştiriyoruz
            merged_df[col] = merged_df[col + "_x"].combine_first(merged_df[col + "_y"])
            # _x ve _y sütunlarını kaldırıyoruz
            merged_df = merged_df.drop(columns=[col + "_x", col + "_y"])

    # Eski verileri (urun'e ait olan satırları) çıkarıyoruz
    data_without_urun = data.drop(index=urun)

    # Yeni verileri ekliyoruz
    data = pd.concat([data_without_urun, merged_df])

    # Data'yı index'e göre sıralıyoruz
    data = data.sort_index()

    return data



In [281]:

import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    pages = ["https://www.carrefoursa.com/pirinc/c/1134?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]

    
    carrefour_data = []
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  = [
    "https://www.migros.com.tr/arama?q=pirin%C3%A7&sayfa=1&kategori=5&sirala=indirim-yuzdesine-gore"
]


# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Pirinç"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()





Scraping URL: https://www.migros.com.tr/arama?q=pirin%C3%A7&sayfa=1&kategori=5&sirala=indirim-yuzdesine-gore
Product: Yayla Uzun Tane Pirinç 2 Kg, Price: 99.95 TRY
Product: Hasata Gönen Baldo Pirinç 2 Kg, Price: 215.95 TRY
Product: Migros Jasmine Pirinç 1000 G, Price: 44.5 TRY
Product: Migros Baldo Pirinç 1000 G, Price: 67.5 TRY
Product: Anadolu Lezzetleri Sarıköy Gönen Baldo Pirinç 1 Kg, Price: 87.9 TRY
Product: Migros Baldo Pirinç 2500 G, Price: 162.5 TRY
Product: Migros Kırık Pirinç 1000 G, Price: 21.9 TRY
Product: Migros Osmancık Pirinç 2500 G, Price: 117.5 TRY
Product: Migros Osmancık Pirinç 1000 G, Price: 51.95 TRY
Product: Migros Pilavlık İthal Pirinç 1000 G, Price: 31.5 TRY
Product: Yayla Gurme Siyah Pirinç 500 G, Price: 63.95 TRY
Product: Yayla Gurme Jasmine Pirinç 500 G, Price: 43.95 TRY
Product: Reis Osmancık Pirinç 1 Kg, Price: 124.95 TRY
Product: Reis Royal Jasmine Pirinç (Premium) 500 G, Price: 104.95 TRY
Product: Yayla Kırık Pirinç 1 Kg, Price: 31.95 TRY
Product: Ekin Ba

In [282]:
data=veriekle("Pirinç",data,urunler_df)

In [283]:
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    pages = ["https://www.carrefoursa.com/bulgur/c/1142?q=%3AbestSeller%3AinStockFlag%3Atrue","https://www.carrefoursa.com/search/?q=un%3AbestSeller%3AinStockFlag%3Atrue&text=un#","https://www.carrefoursa.com/search?q=bu%C4%9Fday%3AbestSeller%3AinStockFlag%3Atrue%3AproductPrimaryCategoryCode%3A1152"]

    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  = ["https://www.migros.com.tr/bulgur-c-426?sayfa=1&sirala=indirim-yuzdesine-gore","https://www.migros.com.tr/bulgur-c-426?sayfa=2&sirala=indirim-yuzdesine-gore","https://www.migros.com.tr/bugday-c-2793?sayfa=1&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/sade-un-c-289b?sayfa=1&sirala=indirim-yuzdesine-gore","https://www.migros.com.tr/bugday-c-2793?sayfa=1&sirala=indirim-yuzdesine-gore"]

# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Un ve Diğer Tahıllar"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()





Scraping URL: https://www.migros.com.tr/bulgur-c-426?sayfa=1&sirala=indirim-yuzdesine-gore
Product: Filiz Mardin Köftelik İnce Bulgur 800 G, Price: 48.95 TRY
Product: Filiz Mardin Pilavlık Bulgur 800 G, Price: 48.95 TRY
Product: M Life Bio Organik Pilavlık Bulgur 1 Kg, Price: 42.9 TRY
Product: M Life Bio Organik Pilavlık Bulgur 1000 G, Price: 42.9 TRY
Product: Anadolu Lezzetleri Amik Ovası Karakılçık Pilavlık Bulgur 500 G, Price: 52.95 TRY
Product: Migros İri Pilavlık Bulgur 1000 G, Price: 26.5 TRY
Product: Anadolu Lezzetleri Amik Ovası Firik 500 G, Price: 70.95 TRY
Product: Torku Köftelik Bulgur 1 Kg, Price: 41.95 TRY
Product: Yayla Gurme Fit Greçkalı Bulgur 360 G, Price: 38.95 TRY
Product: Yayla Gurme Fit Nohutlu Bulgur 360 G, Price: 40.95 TRY
Product: Torku Pilavlık Bulgur 1 Kg, Price: 41.95 TRY
Product: Yayla Gurme Fit Domatesli Bulgur 360 G, Price: 40.95 TRY
Product: Yayla Gurme Fit Kırmızı Kinoalı Bulgur 360 G, Price: 46.95 TRY
Product: Reis Firik Bulgur 1000 Gr, Price: 119.95 TR

In [284]:

data=veriekle("Un ve diğer tahıllar",data,urunler_df)

In [285]:
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros
def scrape_migros_products(base_url, total_pages):
    product_data = []
    
    for page in range(1, total_pages + 1):
        url = f"{base_url}{page}"
        print(f"Scraping page {page}: {url}")
        driver.get(url)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour
def scrape_carrefour_products():
    pages=["https://www.carrefoursa.com/search/?text=ekmek"]
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define base URL and total pages to scrape for Migros
migros_base_url = "https://www.migros.com.tr/arama?q=ekmek&sayfa="
migros_total_pages = 1

# Scrape Migros products
migros_data = scrape_migros_products(migros_base_url, migros_total_pages)

# Scrape Carrefour products
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)


driver.quit()

urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Ekmek"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()





Scraping page 1: https://www.migros.com.tr/arama?q=ekmek&sayfa=1
Product: İhe Tam Buğday Ekmeği 400 G, Price: 40.0 TRY
Product: Uno Premium Çok Tahıllı ve Siyez Buğdaylı Ekmek 350 G, Price: 52.5 TRY
Product: İhe Tuz İlave Edilmemiş Tam Buğday Unlu Ekmek 400 G, Price: 19.0 TRY
Product: İyilik Tam Buğday Ekmeği 350 G, Price: 25.5 TRY
Product: İhe Tost Ekmeği 670G, Price: 45.0 TRY
Product: İhe Altınçerez Fındıklı Ve Üzümlü Ekmek 60 G, Price: 7.5 TRY
Product: Uno Denge Tambuğdaylı Ve Çiya Tohumlu Ekmek 350 G, Price: 53.95 TRY
Product: İhe Tam Buğday Unlu Sandviç 400 G, Price: 25.0 TRY
Product: Uno Premium Çavdarlı ve Karabuğdaylı Ekmek 350 G, Price: 52.5 TRY
Product: İyilik Tost Ekmeği 670 G, Price: 35.5 TRY
Product: İhe Ekşi Mayalı Cıabatta Sandviç Ekmeği 4X80 G, Price: 26.0 TRY
Product: İhe Zeytinli Kekikli Altın Çörek 50 G, Price: 7.5 TRY
Product: Unabella Ponçik Ekmek 200 G, Price: 100.0 TRY
Product: Sofra Ekmek Adet, Price: 9.9 TRY
Product: Kepekli Ekmek Adet, Price: 29.9 TRY
Product:

In [286]:
data=veriekle("Ekmek",data,urunler_df)

In [287]:
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros
def scrape_migros_products(base_url, total_pages):
    product_data = []
    
    for page in range(1, total_pages + 1):
        url = f"{base_url}{page}"
        print(f"Scraping page {page}: {url}")
        driver.get(url)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour
def scrape_carrefour_products():
    pages = ["https://www.carrefoursa.com/corekler/c/1405?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All"]
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define base URL and total pages to scrape for Migros
migros_base_url = "https://www.migros.com.tr/sabah-sicaklari-c-456?sayfa="
migros_total_pages = 1

# Scrape Migros products
migros_data = scrape_migros_products(migros_base_url, migros_total_pages)

# Scrape Carrefour products
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data+carrefour_data
product_df = pd.DataFrame(all_data)


driver.quit()

urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Diğer fırıncılık ürünleri (bisküvi, kek, kraker, yufka, baklava vb.)"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()




Scraping page 1: https://www.migros.com.tr/sabah-sicaklari-c-456?sayfa=1
Product: Simit, Price: 12.5 TRY
Product: Cafe Poğaça Peynirli, Price: 16.0 TRY
Product: Cafe Su Börek Peynirli, Price: 199.9 TRY
Product: Cafe Poğaça Kaşarlı, Price: 18.0 TRY
Product: Cafe Rulo Börek Patatesli, Price: 20.0 TRY
Product: Cafe Rulo Börek Kıymalı, Price: 20.0 TRY
Product: Çikolatalı Açma 110 G (Yeni), Price: 20.0 TRY
Product: Talaş Böreği Peynirli 150 G(yeni), Price: 20.0 TRY
Product: Pizza İtalyan Dılım Ad. 170G, Price: 94.9 TRY
Product: Sade Poğaça Adet, Price: 16.0 TRY
Product: Peynirli Poğaça Adet, Price: 16.0 TRY
Product: Unistanbul Paçanga Böreği 180 G, Price: 139.95 TRY
Product: Sade Açma Adet, Price: 18.0 TRY
Product: Su Böreği Kg, Price: 199.9 TRY
Product: Soğuk Sandviç Peynirli Ad 180 G (Yeni), Price: 79.9 TRY
Product: Talaş Böreği Kıymalı Adet, Price: 20.0 TRY
Product: Cafe Poğaça Patatesli, Price: 16.0 TRY
Product: Kaşarlı Poğaça 80 G, Price: 18.0 TRY
Product 1: Sokak Simit Adet, Price: 12

In [288]:
data=veriekle("Diğer fırıncılık ürünleri (bisküvi, kek, kraker, yufka, baklava vb.)",data,urunler_df)

In [289]:
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros
def scrape_migros_products(base_url, total_pages):
    product_data = []
    
    for page in range(1, total_pages + 1):
        url = f"{base_url}{page}"
        print(f"Scraping page {page}: {url}")
        driver.get(url)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour
def scrape_carrefour_products():
    pages = ["https://www.carrefoursa.com/makarna/c/1122?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define base URL and total pages to scrape for Migros
migros_base_url = "https://www.migros.com.tr/normal-makarna-c-2780?sayfa="
migros_total_pages = 5

# Scrape Migros products
migros_data = scrape_migros_products(migros_base_url, migros_total_pages)

# Scrape Carrefour products
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)


driver.quit()

urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Makarna çeşitleri"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()



Scraping page 1: https://www.migros.com.tr/normal-makarna-c-2780?sayfa=1
Product: Pastavilla Kalem Makarna 500 G, Price: 27.95 TRY
Product: Pastavilla Burgu Makarna 500 G, Price: 27.95 TRY
Product: Pastavilla Spaghetti Makarna 500 G, Price: 27.95 TRY
Product: Pastavilla Kuskus Makarna 500 G, Price: 27.95 TRY
Product: Migros Arpa Şehriye 500 G, Price: 10.75 TRY
Product: Migros Burgu Makarna 500 G, Price: 10.75 TRY
Product: Pastavilla Dirsek Makarna 500 G, Price: 27.95 TRY
Product: Pastavilla Mantı Makarna 500 G, Price: 27.95 TRY
Product: Filiz Boncuk Makarna 500 G, Price: 22.75 TRY
Product: Migros Yüksük Makarna 500 G, Price: 10.75 TRY
Product: Pastavilla Kelebek Makarna 500 G, Price: 27.95 TRY
Product: Filiz Arpa Şehriye 500 G, Price: 22.75 TRY
Product: Filiz Spagetti (Çubuk) Makarna 500 G, Price: 22.75 TRY
Product: Migros Uzun İnce Makarna 500 G, Price: 10.75 TRY
Product: Filiz Fiyonk Makarna 500 G, Price: 22.75 TRY
Product: Migros Spagetti Makarna 500 G, Price: 10.75 TRY
Product: Pas

In [290]:
data=veriekle("Makarna çeşitleri",data,urunler_df)

In [291]:
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros
def scrape_migros_products(base_url, total_pages):
    product_data = []
    
    for page in range(1, total_pages + 1):
        url = f"{base_url}{page}"
        print(f"Scraping page {page}: {url}")
        driver.get(url)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour
def scrape_carrefour_products():
        url = "https://www.carrefoursa.com/musli-hububat-urunleri/c/1378?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"

    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
        return carrefour_data

# Define base URL and total pages to scrape for Migros
migros_base_url = "https://www.migros.com.tr/kahvaltilik-gevrek-c-422?sayfa="
migros_total_pages = 4

# Scrape Migros products
migros_data = scrape_migros_products(migros_base_url, migros_total_pages)

# Scrape Carrefour products
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)


driver.quit()

urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Kahvaltılık tahıl ürünleri"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()



Scraping page 1: https://www.migros.com.tr/kahvaltilik-gevrek-c-422?sayfa=1
Product: Eti Lifalif Yulaf Ezmesi 500 G, Price: 47.95 TRY
Product: Eti Lifalif İnce Öğütülmüş Yulaf Ezmesi 350 G, Price: 47.95 TRY
Product: S.S Tire Organik Yulaf Ezmesi 500 G, Price: 30.95 TRY
Product: Nestlé Nesquik Harfler Kakaolu Buğday ve Mısır Gevreği 310g, Price: 87.95 TRY
Product: Nestlé Nesfit Çikolatalı Tam Tahıl ve Pirinç Gevreği 400g, Price: 118.95 TRY
Product: Nestle Gold Corn Flakes Tam Tahıl Mıs.gevreği 400G, Price: 92.5 TRY
Product: Fropie Badem&Vanilya Probiyotik Granola 200 G, Price: 109.95 TRY
Product: Migros Granola Yulaflı Kakaolu 250 G, Price: 45.45 TRY
Product: Rıchland Granola Yulaflı Ballı ve Keten Tohumu 330G, Price: 69.95 TRY
Product: Eti Lifalif Granola Kuru Vişneli, Kakao Parçacıklı, Fındıklı 200 g, Price: 94.95 TRY
Product: Moms Granola Muz Ceviz 360 G, Price: 189.9 TRY
Product: Moms Granola Çilek Chia 360 G, Price: 209.9 TRY
Product: Crunch Tam Tahıl Ve Pirinç Gevreği Kutu 280 G, 

In [292]:
data=veriekle("Kahvaltılık tahıl ürünleri",data,urunler_df)

In [293]:

import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    pages = ["https://www.carrefoursa.com/dana/c/1046?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]


    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  = ["https://www.migros.com.tr/dana-eti-c-3fa?sayfa=1&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/dana-eti-c-3fa?sayfa=2&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/dana-eti-c-3fa?sayfa=3&sirala=indirim-yuzdesine-gore"]


# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Dana eti"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()





Scraping URL: https://www.migros.com.tr/dana-eti-c-3fa?sayfa=1&sirala=indirim-yuzdesine-gore
Product: Uzman Kasap Misket Köfte 400 G, Price: 209.95 TRY
Product: Erşan Pişmiş Dana Döner 250 G, Price: 159.95 TRY
Product: Uzman Kasap Dana Hamburger Köfte 330 G, Price: 179.95 TRY
Product: Dana But Tas Kebaplık Et Çiftlik Kg, Price: 619.95 TRY
Product: Dana But Tas Kebaplık Et Yöresel Kg, Price: 649.95 TRY
Product: Tire Süt Şiş Köfte 250 G, Price: 275.0 TRY
Product: Taze Otlu Dana Antrikot Kg, Price: 969.95 TRY
Product: Dana But Sotelik Et Yöresel Kg, Price: 649.95 TRY
Product: Dana Çiğ Köftelik Et Yöresel Kg, Price: 619.95 TRY
Product: Dana Fırınlık Kaburga Kg, Price: 489.95 TRY
Product: Dana Kuzu Kıymalık Kg, Price: 419.95 TRY
Product: Dana Biftek Yöresel Kg, Price: 645.95 TRY
Product: Dana Stragonofluk Et Yöresel Kg, Price: 649.95 TRY
Product: Dana Antrikot Yöresel Kg, Price: 999.95 TRY
Product: Dana Bonfile Kg, Price: None TRY
Product: Dana Haşlamalık Et Kg, Price: 539.95 TRY
Product: D

In [294]:
data=veriekle("Dana eti",data,urunler_df)

In [295]:
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    pages = ["https://www.carrefoursa.com/kuzu/c/1054?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]


    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  = ["https://www.migros.com.tr/kuzu-eti-c-3fb?sayfa=1&sirala=indirim-yuzdesine-gore"]


# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Kuzu eti"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()






Scraping URL: https://www.migros.com.tr/kuzu-eti-c-3fb?sayfa=1&sirala=indirim-yuzdesine-gore
Product: Kuzu İncik Kg, Price: 585.95 TRY
Product: Kuzu Fırınlık Kol Kg (Kemikli, Bütün Kol), Price: 569.95 TRY
Product: Kuzu Gerdan Kg (Kemikli, Dilimli), Price: 419.95 TRY
Product: Kuzu Sac Kavurmalık Kg, Price: 499.95 TRY
Product: Kuzu Yemeklik Kuşbaşı Kg, Price: 665.95 TRY
Product: Kuzu Sotelik Et Kg, Price: 784.95 TRY
Product: Kuzu Yarım Kol Kg (Orta Kemikli, İnciksiz Kol), Price: 669.95 TRY
Product: Kuzu Şişlik But Kg (Orta Kemikli, İnciksiz Bütün But), Price: 679.95 TRY
Product: Kuzu Kol Kg (Kemikli, Bütün Kol), Price: 569.95 TRY
Product: Kuzu But Kg (Kemikli, Bütün But), Price: 579.95 TRY
Product: Kuzu Kuşbaşı Kg, Price: 765.95 TRY
Product: Kuzu Külbastı Kg (But Eti, Kemiksiz, Dilimli), Price: 725.95 TRY
Product: Uzman Kasap Kuzu Lokum Şiş Skinpack, Price: 959.95 TRY
Product: Uzman Kasap Kuzu Sac Kavurmalık 400 G, Price: 179.95 TRY
Product: Kuzu Pirzola Kg, Price: 899.95 TRY
Product 1: 

In [296]:


data=veriekle("Kuzu eti",data,urunler_df)

In [297]:

import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    pages = ["https://www.carrefoursa.com/pilic/c/1061?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]



    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  = ["https://www.migros.com.tr/pilic-c-3fe?sayfa=1&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/pilic-c-3fe?sayfa=2&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/pilic-c-3fe?sayfa=3&sirala=indirim-yuzdesine-gore"]


# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Tavuk eti"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()








Scraping URL: https://www.migros.com.tr/pilic-c-3fe?sayfa=1&sirala=indirim-yuzdesine-gore
Product: Lezita Izgara Dilim Piliç Fileto Pesto Soslu 200G, Price: 62.95 TRY
Product: Lezita Izgara Dilim Piliç Fileto Chili Soslu 200G, Price: 62.95 TRY
Product: Şenpiliç Piliç Kasap Köfte 300 G, Price: 49.95 TRY
Product: Keskinoğlu Acılı Piliç Kebap 400 G, Price: 52.5 TRY
Product: Banvit Piliç Schnitzel 700 G, Price: 97.95 TRY
Product: Pınar Piliç Şnitzel 415 G, Price: 179.9 TRY
Product: Gedik Dondurulmuş Poşetli Piliç Izgara Tava 1 Kg, Price: 366.95 TRY
Product: Gedik Dondurulmuş Poşetli Piliç Pirzola 1 Kg, Price: 242.95 TRY
Product: Gedik Dondurulmuş Poşetli Piliç Baget 1 Kg, Price: 131.95 TRY
Product: Gedik Dondurulmuş Poşetli Piliç Kanat 1 Kg, Price: 322.95 TRY
Product: Keskinoğlu Piliç Döner 500 G, Price: 154.95 TRY
Product: Lezita Acılı Çıtır Jumbo Fileto 300 G, Price: 106.95 TRY
Product: İmren Piliç Izgara Kanat Kg, Price: 229.0 TRY
Product: İmren Piliç Pirzola Kg, Price: 159.0 TRY
Produc

In [298]:
data=veriekle("Tavuk eti",data,urunler_df)

In [299]:
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data



# Define the list of pages to scrape from Migros
migros_pages  = [
    "https://www.migros.com.tr/arama?q=sakatat&sayfa=1&kategori=3&sirala=indirim-yuzdesine-gore",
    "https://www.migros.com.tr/hindi-c-3ff?sayfa=1&sirala=indirim-yuzdesine-gore"
]

# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)



all_data = migros_data 
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Diğer etler ve yenilebilir sakatatlar"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()









Scraping URL: https://www.migros.com.tr/arama?q=sakatat&sayfa=1&kategori=3&sirala=indirim-yuzdesine-gore
Product: Kuzu Yürek Kg, Price: 435.95 TRY
Product: Dana Yürek Kg, Price: 279.95 TRY
Product: Kuzu Karaciğer Kg, Price: 765.95 TRY
Product: Kuzu Böbrek Kg, Price: 495.95 TRY
Product: Dana Ciğer Kg, Price: 659.95 TRY
Product: Dana Böbrek Kg, Price: 129.95 TRY
Product: Dana Kuyruk (Pöç) Kg, Price: 425.95 TRY
Product: Dana İşkembe Kg, Price: 229.95 TRY
Product: Dana Billur Kg, Price: 52.95 TRY
Product: Dana Dil Kg, Price: 449.95 TRY
Product: Kuzu Dalak, Price: 129.95 TRY
Product: Koç Yumurtası Kg, Price: 88.95 TRY
Product: Kuzu İşkembe Kg, Price: 149.95 TRY
Product: Gurvita Paça Çorbası 660 Ml, Price: 362.95 TRY
Product: Gurvita İlikli Kemik Sulu Mercimek Çorbası 660 Ml, Price: 242.95 TRY
Product: Eral Dana Kokoreç (Dilimlenmiş-pişmiş) 200 G, Price: 168.95 TRY
Product: Eral Dana Kuzu Karışık Kokoreç 200 G, Price: 192.95 TRY
Product: Gurvita Kuzu İlikli Kemik Suyu 320 Ml, Price: 132.95 T

In [300]:
data=veriekle("Diğer etler ve yenilebilir sakatatlar",data,urunler_df)

In [301]:

import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    pages = ["https://www.carrefoursa.com/sarkuteri/c/1070?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All"]



    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  = ["https://www.migros.com.tr/et-sarkuteri-c-6b?sayfa=1&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/et-sarkuteri-c-6b?sayfa=2&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/et-sarkuteri-c-6b?sayfa=3&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/et-sarkuteri-c-6b?sayfa=4&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/et-sarkuteri-c-6b?sayfa=5&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/et-sarkuteri-c-6b?sayfa=6&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/et-sarkuteri-c-6b?sayfa=7&sirala=indirim-yuzdesine-gore",
]


# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Şarküteri ürünleri (sucuk, sosis, salam vb.)"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()









Scraping URL: https://www.migros.com.tr/et-sarkuteri-c-6b?sayfa=1&sirala=indirim-yuzdesine-gore
Product: Erşan Dilimli Çemeni Sıyrılmış Dana Pastırma 110G, Price: 315.0 TRY
Product: Namet Piliç Füme 150 G, Price: 82.9 TRY
Product: Torku Fıstıklı Salam 250 G, Price: 194.95 TRY
Product: Torku Etli Macar Salam 250G, Price: 189.95 TRY
Product: Pınar Aç Bitir Hindi Salam Taze Dilimlenmiş 10 Dilim 75 G, Price: 30.9 TRY
Product: Pınar 10'Lu Ekonomik Sosis 430 G, Price: 359.0 TRY
Product: Torku Dilimli Salam 60 G, Price: 43.95 TRY
Product: Torku Dilimli Fıstıklı Salam 50 G, Price: 43.95 TRY
Product: Polonez Fit Yaşam XL Hindi Füme 150 G, Price: 89.9 TRY
Product: Polonez Zeytinyağlı Kekikli Hindi Füme 150 G, Price: 89.9 TRY
Product: Özlem Dana Kangal Sucuk 220 G, Price: 215.0 TRY
Product: Şahin Klasik Kayseri Çemeni 200 G, Price: 59.9 TRY
Product: Pınar Aç Bitir Hindi Salam Büyük Dilim 60 G, Price: 26.9 TRY
Product: Pınar Salam Aç Bitir Hindi Fıstıklı Salam 50 G, Price: 26.9 TRY
Product: Polone

In [302]:
data=veriekle("Şarküteri ürünleri (sucuk, sosis, salam vb.)",data,urunler_df)

In [303]:
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    pages = ["https://www.carrefoursa.com/baliklar/c/1099?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]



    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  = ["https://www.migros.com.tr/balik-deniz-urunleri-c-6a?sayfa=1&markalar=492&sirala=indirim-yuzdesine-gore"
]


# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Taze balık"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()









Scraping URL: https://www.migros.com.tr/balik-deniz-urunleri-c-6a?sayfa=1&markalar=492&sirala=indirim-yuzdesine-gore
Product: Levrek 600/800 Kg, Price: 399.95 TRY
Product: Levrek 400/600 Kg, Price: 349.95 TRY
Product: Santa Bremor Somon İmitasyon Havyar 230 G, Price: 309.95 TRY
Product: Kefal Orta Kg, Price: 169.95 TRY
Product: Lüfer Kg, Price: None TRY
Product: Hamsi Kg (Temizlenmeden Gönderilecektir.), Price: 249.95 TRY
Product: Jumbo Karides Orta Kg, Price: 599.95 TRY
Product: Çipura 400/600 Kg, Price: 379.95 TRY
Product: Çipura 600/800 Kg(m), Price: 399.95 TRY
Product: Kaya Levrek Toprak 1000/2000 Kg (M), Price: 399.95 TRY
Product: Lahoz Orta Kg, Price: None TRY
Product: Sazan Orta Kg (M), Price: 189.96 TRY
Product: Akdeniz Lokum Kg (M), Price: 199.95 TRY
Product: Fener Balığı Orta Kg (M), Price: 449.95 TRY
Product: Karides Cimcim Kg, Price: 249.95 TRY
Product: Tavuk Balığı Orta Kg, Price: 499.95 TRY
Product: Kerevit Kg (M), Price: 109.95 TRY
Product: İstavrit Kıraca Kg (M), Price:

In [304]:
data=veriekle("Taze balık",data,urunler_df)

In [305]:

import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    pages = ["https://www.carrefoursa.com/paketli-urunler/c/1068?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]



    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  = ["https://www.migros.com.tr/balik-deniz-urunleri-c-6a?sayfa=2&markalar=539,6344,221,425,170&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/balik-deniz-urunleri-c-6a?sayfa=1&markalar=539,6344,221,425,170&sirala=indirim-yuzdesine-gore"
]



# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Konserve edilmiş veya işlenmiş balık"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()










Scraping URL: https://www.migros.com.tr/balik-deniz-urunleri-c-6a?sayfa=2&markalar=539,6344,221,425,170&sirala=indirim-yuzdesine-gore
Product: Balık Dünyası Dondurulmuş Kalamar Tava 115G, Price: 114.95 TRY
Product: Kocaman Et Karides 200/300 500 G, Price: 259.95 TRY
Product: Pınar Çıtır Balık 400 G, Price: 237.95 TRY
Product: Pınar Fish Finger 240 G, Price: 157.95 TRY
Product: Dardanel Kolay Pişir Karidesli Domates Soslu Penne 300G, Price: 149.95 TRY
Product: Dardanel Kolay Pişir Tereyağlı Jumbo Karides 250 G, Price: 239.9 TRY
Product: Balık Dünyası Dondurulmuş Midye Tava 150G, Price: 94.95 TRY
Product: Dardanel Kolay Pişir Karidesli Pesto Soslu Linguine 300G, Price: 149.95 TRY
Product: Dardanel Fish Fingers Gold 470 G, Price: 199.95 TRY
Product: Migros Karides 400 G, Price: 259.9 TRY
Product: Balık Dünyası Dondurulmuş Et Karides 250 G, Price: 229.95 TRY
Scraping URL: https://www.migros.com.tr/balik-deniz-urunleri-c-6a?sayfa=1&markalar=539,6344,221,425,170&sirala=indirim-yuzdesine-gore

In [306]:
data=veriekle("Konserve edilmiş veya işlenmiş balık",data,urunler_df)

In [307]:

import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    pages = ["https://www.carrefoursa.com/sut/c/1311?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All"]



    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  = ["https://www.migros.com.tr/sut-c-6c?sayfa=1&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/sut-c-6c?sayfa=2&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/sut-c-6c?sayfa=3&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/sut-c-6c?sayfa=4&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/sut-c-6c?sayfa=5&sirala=indirim-yuzdesine-gore"
]

# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Süt"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()










Scraping URL: https://www.migros.com.tr/sut-c-6c?sayfa=1&sirala=indirim-yuzdesine-gore
Product: Danone Disney Muzlu Süt 6x180 Ml, Price: 93.95 TRY
Product: Danone Disney Çilekli Süt 6X180 Ml, Price: 93.95 TRY
Product: Danone Disney Kakaolu Süt 6x180 Ml, Price: 93.95 TRY
Product: Danone Disney Kakaolu Süt 9X180 Ml, Price: 129.95 TRY
Product: Danone Disney Kakaolu Süt 180 Ml, Price: 18.75 TRY
Product: Danone Disney Çilekli Süt 180 Ml, Price: 18.75 TRY
Product: Danone Disney Muzlu Süt 180 Ml, Price: 18.75 TRY
Product: Danone Doğal Süt 180Ml, Price: 18.75 TRY
Product: Fomilk Badem 200 Ml, Price: 29.95 TRY
Product: Fomilk Badem 1 L, Price: 115.0 TRY
Product: Fomilk Badem Şekersiz 1 L, Price: 115.0 TRY
Product: Alpro Yulaf İçeceği 500 Ml, Price: 84.95 TRY
Product: Alpro Bademli İçecek 250 Ml, Price: 47.95 TRY
Product: Torku Çiftçi Sütü Günlük 1 L, Price: 56.5 TRY
Product: Nesquik Kakaolu Süt 6x180 Ml, Price: 119.95 TRY
Product: Sek Karamel Macchiato 330 Ml, Price: 34.95 TRY
Product: Sek Kahv

In [308]:
data=veriekle("Süt",data,urunler_df)

In [309]:

import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    pages = ["https://www.carrefoursa.com/yogurt/c/1389?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All"]



    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  = ["https://www.migros.com.tr/yogurt-c-6e?sayfa=1&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/yogurt-c-6e?sayfa=2&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/yogurt-c-6e?sayfa=3&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/yogurt-c-6e?sayfa=4&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/sutlu-tatli-krema-c-72?sayfa=1&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/sutlu-tatli-krema-c-72?sayfa=2&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/sutlu-tatli-krema-c-72?sayfa=3&sirala=indirim-yuzdesine-gore"

]


# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Diğer süt ürünleri (yoğurt, hazır sütlü tatlı vb.)"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()











Scraping URL: https://www.migros.com.tr/yogurt-c-6e?sayfa=1&sirala=indirim-yuzdesine-gore
Product: Danone Meyveli Yoğurt Çilekli 400G, Price: 55.95 TRY
Product: Danone Meyveli Yoğurt Orman Meyveli 400G, Price: 55.95 TRY
Product: Danone Activia Sade Yoğurt 400 G, Price: 54.95 TRY
Product: Activia Shot+ Nar Ve Çilek 80 Ml, Price: 19.0 TRY
Product: Activia Shot+ Kara Mürver Ve Yaban Mersinli 80 Ml, Price: 19.0 TRY
Product: Activia Shot Elma Yulaf 80 Ml, Price: 19.0 TRY
Product: Eker Kaymaklı Tava Yoğurt 1 Kg, Price: 79.95 TRY
Product: Torku Yoğurt 1.5 Kg, Price: 84.95 TRY
Product: Activia Shot X2 Probiyotik Sade 4x80ml, Price: 64.95 TRY
Product: Pınar Yoğurt 2 Kg, Price: 120.0 TRY
Product: Vivo Yoğurt Mayası 4x1G, Price: 47.5 TRY
Product: Sütaş Süzme Yoğurt 750 G, Price: 92.5 TRY
Product: Eker İncirli Yulaflı Probiyotik Yoğurt 125 G, Price: 18.95 TRY
Product: Pınar Yoğurt 150 G, Price: 18.95 TRY
Product: Eker Dip Cips Domates Soslu Yoğurt Sos 125 G, Price: 22.95 TRY
Product: Sek Çiftlik K

In [310]:
data=veriekle("Diğer süt ürünleri (yoğurt, hazır sütlü tatlı vb.)",data,urunler_df)

In [311]:

import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    pages = ["https://www.carrefoursa.com/peynir/c/1318?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All"]



    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  =  ["https://www.migros.com.tr/peynir-c-6d?sayfa=1&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/peynir-c-6d?sayfa=2&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/peynir-c-6d?sayfa=3&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/peynir-c-6d?sayfa=4&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/peynir-c-6d?sayfa=5&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/peynir-c-6d?sayfa=6&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/peynir-c-6d?sayfa=7&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/peynir-c-6d?sayfa=8&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/peynir-c-6d?sayfa=9&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/peynir-c-6d?sayfa=10&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/peynir-c-6d?sayfa=11&sirala=indirim-yuzdesine-gore",

]



# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Peynir"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()












Scraping URL: https://www.migros.com.tr/peynir-c-6d?sayfa=1&sirala=indirim-yuzdesine-gore
Product: Bahçıvan Grana Padano Peyniri 200 G, Price: 399.9 TRY
Product: Fomılk Bitkisel Bazlı Zahterli Sürülebilir Kıvamda Yiyecek 125 G, Price: 67.45 TRY
Product: Fomilk Bitkisel Bazlı Dereotlu Sürülebilir Kıvamda Yiyecek 125 G, Price: 67.45 TRY
Product: Bahçıvan Laktozsuz Labne 200 G, Price: 61.5 TRY
Product: Bahçıvan Hellim Peyniri 225 G, Price: 131.95 TRY
Product: Bahçıvan Yarım Yağlı Laktozsuz Süzme Peynir 500G, Price: 117.5 TRY
Product: Bahçıvan Tam Yağlı Probiyotik Süzme Beyaz Peynir 500 G, Price: 117.5 TRY
Product: Bahçıvan Labne 200 G, Price: 58.5 TRY
Product: Kiri Labne Peyniri 150 G, Price: 51.95 TRY
Product: Torku Tam Yağlı Taze Beyaz Peynir 900 G, Price: 223.0 TRY
Product: Milgo Taze Sürülebilir Hurmalı Peynir 200 G, Price: 79.95 TRY
Product: Milgo Taze Sürülebilir Laktozsuz Peynir 200 Gr, Price: 79.95 TRY
Product: Milgo Taze Sürülebilir Sade Peynir 200 G, Price: 79.95 TRY
Product: La

In [312]:
data=veriekle("Peynir",data,urunler_df)

In [313]:

import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    pages = ["https://www.carrefoursa.com/yumurta/c/1349?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]



    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  =  ["https://www.migros.com.tr/yumurta-c-70?sayfa=1&sirala=indirim-yuzdesine-gore"

]





# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Yumurta"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()













Scraping URL: https://www.migros.com.tr/yumurta-c-70?sayfa=1&sirala=indirim-yuzdesine-gore
Product: Gürata Ordu Yumurta Üreticileri Birliği Organik 10'lu M Boy (53-62g), Price: 85.9 TRY
Product: Egem Organik Yumurta 6'lı M Boy (53-63 G), Price: 54.9 TRY
Product: Yumurtacım 20'li Xl Yumurta (73 G Ve Üzeri), Price: 129.95 TRY
Product: Yumurtacım Omega 3 10'Lu M Boy Yumurta (53-62 G), Price: 68.95 TRY
Product: Yumurtacım 30'lu L Boy Yumurta (63-72 G), Price: 156.5 TRY
Product: Güres M - Orta Boy Yumurta 30'lu ( 53 - 62 G), Price: 125.0 TRY
Product: Coskun Organik M Boy Yumurta 15'li (53-63 G), Price: 112.95 TRY
Product: Yumurtacım 15'li L Boy Yumurta (63-72 G), Price: 72.5 TRY
Product: Naturaköy Selenyum 15'li Yumurta M - Orta Boy 53-62 G, Price: 85.95 TRY
Product: Yumurtacım 6'lı M Boy Yumurta (53-62 G), Price: 39.95 TRY
Product: Coşkun 20'li L Boy Yumurta (63-72 G), Price: 91.95 TRY
Product: Güres Bıldırcın Yumurta 12'li, Price: 42.9 TRY
Product: Yumurtacım Organik 10'Lu M Boy Yumurta (

In [314]:
data=veriekle("Yumurta",data,urunler_df)

In [315]:

import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    pages = ["https://www.carrefoursa.com/tereyag/c/1350?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]



    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  =  ["https://www.migros.com.tr/tereyagi-c-413?sayfa=1&sirala=indirim-yuzdesine-gore"

]





# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Tereyağı"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()


















Scraping URL: https://www.migros.com.tr/tereyagi-c-413?sayfa=1&sirala=indirim-yuzdesine-gore
Product: Özışık Tereyağı 1 Kg, Price: 375.0 TRY
Product: Yörük Çiftliği Ghee Sade Yağ 300 G, Price: 289.0 TRY
Product: Torku Tereyağı 1 Kg, Price: 490.0 TRY
Product: Laz Kızı Tereyağı 250 G, Price: 129.95 TRY
Product: İçim Tereyağı 500 G, Price: 235.95 TRY
Product: S.S Tire Süt Kooperatifi Sade Yağ 225 G, Price: 188.95 TRY
Product: Pınar Tereyağı Folyo 200 G, Price: 99.95 TRY
Product: Sek Tereyağı 1 Kg, Price: 450.0 TRY
Product: Trakya Çiftliği Vegan Yayık 200 G, Price: 79.95 TRY
Product: Pınar Tereyağı 125 G, Price: 68.95 TRY
Product: Efeler Çiftliği Tereyağ 200 G, Price: 178.5 TRY
Product: Sek Tereyağ 100 G, Price: 53.95 TRY
Product: Sek Tereyağ 200 G, Price: 105.0 TRY
Product: Sek Köy Tereyağı 225 G, Price: 112.95 TRY
Product: Hasmandıra Geleneksel Tereyağı 500 G, Price: 239.95 TRY
Product: İçim Tereyağı 225 G, Price: 99.95 TRY
Product: Ariste Tereyağı 500 G, Price: 279.95 TRY
Product: Sütaş

In [316]:
data=veriekle("Tereyağı",data,urunler_df)

In [317]:

import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    pages = ["https://www.carrefoursa.com/margarin/c/1351?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]



    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  =  ["https://www.migros.com.tr/margarin-c-414?sayfa=1&sirala=indirim-yuzdesine-gore"

]






# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Margarin"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()


















Scraping URL: https://www.migros.com.tr/margarin-c-414?sayfa=1&sirala=indirim-yuzdesine-gore
Product: Becel Kase Margarin Zeytinyağlı 500 G, Price: 75.9 TRY
Product: Becel Kase Margarin Klasik 250 G, Price: 35.0 TRY
Product: Becel Kase Margarin Zeytinyağlı 250 G, Price: 39.9 TRY
Product: Becel Kase Margarin Klasik 500 G, Price: 66.5 TRY
Product: Becel Kase Margarin Tereyağı Keyfi 250 G, Price: 35.0 TRY
Product: Becel Kase Margarin Tereyağı Keyfi 500 G, Price: 66.5 TRY
Product: Lurpak Organik Sürülebilir Yağ 250 G, Price: 274.95 TRY
Product: Lurpak Sürülebilir Tuzsuz Tereyağı 250 G, Price: 274.95 TRY
Product: Bizim Kase Margarin 250 Gr, Price: 32.5 TRY
Product: Sana Kase Margarin Ekmeküstü 500 G, Price: 71.9 TRY
Product: Lurpak Sürülebilir Tuzlu Tereyağı 250 G, Price: 274.95 TRY
Product: Sana Paket Margarin Hamurişi 250 G, Price: 33.9 TRY
Product: Sana Kase Margarin Klasik 500 G, Price: 59.9 TRY
Product: Sana Klasik Paket Margarin 4X250 G Saklama Kabı Hediye, Price: 101.95 TRY
Product: 

In [318]:
data=veriekle("Margarin",data,urunler_df)

In [319]:

import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    pages = ["https://www.carrefoursa.com/sivi-yaglar/c/1111?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All"]



    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  = ["https://www.migros.com.tr/aycicek-yagi-c-42d?sayfa=1&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/zeytinyagi-c-433?sayfa=1&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/zeytinyagi-c-433?sayfa=2&sirala=indirim-yuzdesine-gore"

]





# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Sıvı yağlar (zeytinyağı, ayçiçek yağı)"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()



















Scraping URL: https://www.migros.com.tr/aycicek-yagi-c-42d?sayfa=1&sirala=indirim-yuzdesine-gore
Product: Emek Ayçiçek Yağı 5 L Köşeli Pet, Price: 279.95 TRY
Product: Torku Ayçiçek Yağı 5 L Pet, Price: 264.95 TRY
Product: Yudum Zerdeçallı Ayçiçekyağı 1L, Price: 66.0 TRY
Product: Kırlangıç Ayçiçek Yağı 1 L, Price: 77.95 TRY
Product: Abalı Ayçiçek Yağı 1 L, Price: 76.95 TRY
Product: Orkide Ayçiçek Yağı 2 L, Price: 142.95 TRY
Product: Emek Ayçiçek Yağı 2 L, Price: 139.95 TRY
Product: Migros Ayçiçek Yağı 1 L, Price: 61.5 TRY
Product: Migros Ayçiçek Yağı 2 L, Price: 119.0 TRY
Product: Yudum Ayçiçekyağı 5L (Koşeli Pet), Price: 319.5 TRY
Product: Migros Ayçiçek Yağı 5 L Köşeli Pet, Price: 259.0 TRY
Product: Yudum Ayçiçekyağı 1L, Price: 106.95 TRY
Product: Komili Ayçiçek Yağı 1 L, Price: 105.95 TRY
Product: Komili Ayçiçek Yağı 2 L, Price: 183.95 TRY
Product: Yudum Ayçiçekyağı 2L, Price: 184.95 TRY
Product: Komili Ayçiçek Yağı 5 L, Price: 298.95 TRY
Scraping URL: https://www.migros.com.tr/zeyti

In [320]:
data=veriekle("Sıvı yağlar (zeytinyağı, ayçiçek yağı)",data,urunler_df)

In [321]:

import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    
    pages = ["https://www.carrefoursa.com/meyve/c/1015?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All"]



    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  = ["https://www.migros.com.tr/kavun-ve-karpuz-c-3eb?sayfa=1&markalar=492&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/narenciye-c-3ec?sayfa=1&markalar=492&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/yumusak-meyveler-c-3ee?sayfa=1&markalar=492&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/sert-meyveler-c-3ed?sayfa=1&markalar=492&sirala=indirim-yuzdesine-gore",


]





# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Taze meyveler"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()




















Scraping URL: https://www.migros.com.tr/kavun-ve-karpuz-c-3eb?sayfa=1&markalar=492&sirala=indirim-yuzdesine-gore
Product: Kavun Biga Kg, Price: 29.95 TRY
Product: Kavun Sarı, Price: 19.95 TRY
Product: Kavun Kırkağaç Kg, Price: 15.95 TRY
Scraping URL: https://www.migros.com.tr/narenciye-c-3ec?sayfa=1&markalar=492&sirala=indirim-yuzdesine-gore
Product: Portakal Kg, Price: 39.95 TRY
Product: Limon Naturel Kg, Price: 29.95 TRY
Product: Greyfurt Kan Kg, Price: 39.95 TRY
Product: Limon Kg, Price: 32.95 TRY
Product: Portakal Sıkma File Kg, Price: 39.95 TRY
Product: Mandalina Naturel Kg, Price: 39.9 TRY
Scraping URL: https://www.migros.com.tr/yumusak-meyveler-c-3ee?sayfa=1&markalar=492&sirala=indirim-yuzdesine-gore
Product: Şeftali Nektarin Kg, Price: 69.95 TRY
Product: Şeftali Kg, Price: 69.95 TRY
Product: Erik Angelica Kg, Price: 57.95 TRY
Product: Üzüm Siyah Kg, Price: 69.95 TRY
Product: Yaş Zeytin Jumbo Boy Kg, Price: 69.95 TRY
Product: Zeytin Yaş Kg, Price: 59.95 TRY
Product: Üzüm Çekirde

In [322]:
data=veriekle("Taze meyveler",data,urunler_df)

In [323]:

import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    
    pages = ["https://www.carrefoursa.com/kuruyemis/c/1519?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All"]



    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  =  ["https://www.migros.com.tr/arama?q=kuru%20meyve&sayfa=4&kategori=1089&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=kuru%20meyve&sayfa=3&kategori=1089&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=kuru%20meyve&sayfa=2&kategori=1089&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=kuru%20meyve&sayfa=1&kategori=1089&sirala=indirim-yuzdesine-gore",
         

]





# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Kuru meyve ve sert kabuklu yemişler"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()





















Scraping URL: https://www.migros.com.tr/arama?q=kuru%20meyve&sayfa=4&kategori=1089&sirala=indirim-yuzdesine-gore
Product: Çiğ Badem Kg, Price: 629.95 TRY
Product: Hurma Medjoul Kg, Price: 499.95 TRY
Product: Master Farm İç Ceviz 150 G, Price: 97.95 TRY
Product: Siha Kurutulmuş Dolmalık Biber 25'li Paket, Price: 129.95 TRY
Product: Siha Kurutulmuş Dolmalık Patlıcan 25'li Paket, Price: 129.95 TRY
Product: Garliko Kurutulmuş Dolmalık Biber 25'li Paket, Price: 119.95 TRY
Product: Dünya Hayat Kurutulmuş Patlıcan, Price: 119.95 TRY
Product: Tadım Güneşte Kurutulmuş Kayısı 140 G, Price: 69.95 TRY
Product: M Life Çiğ İç Fındık 200 G, Price: 88.9 TRY
Product: Master Farm Kuru Kayısı 140 G, Price: 59.95 TRY
Scraping URL: https://www.migros.com.tr/arama?q=kuru%20meyve&sayfa=3&kategori=1089&sirala=indirim-yuzdesine-gore
Product: Master Farm Çiğ İç Badem 160 G, Price: 102.95 TRY
Product: Peyman Bahçeden Çiğ Karışk Kuruyemiş Ara Öğün 140G, Price: 87.45 TRY
Product: Gojiberry Kg, Price: 639.95 TRY
Pr

In [324]:
data=veriekle("Kuru meyve ve sert kabuklu yemişler",data,urunler_df)

In [325]:

import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    
    pages = ["https://www.carrefoursa.com/sebze/c/1025?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All"]



    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  = ["https://www.migros.com.tr/sebze-c-66?sayfa=1&markalar=492&94=661,664,666,693,676,694,686,696,681,697,691,700,673,705,684,708,712,713,668,721,722,723,724,726,729,730,732,733,735,736,738,727,692,667,743,745,671,689,674,747,749,740,755&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/sebze-c-66?sayfa=2&markalar=492&94=661,664,666,693,676,694,686,696,681,697,691,700,673,705,684,708,712,713,668,721,722,723,724,726,729,730,732,733,735,736,738,727,692,667,743,745,671,689,674,747,749,740,755&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/sebze-c-66?sayfa=3&markalar=492&94=661,664,666,693,676,694,686,696,681,697,691,700,673,705,684,708,712,713,668,721,722,723,724,726,729,730,732,733,735,736,738,727,692,667,743,745,671,689,674,747,749,740,755&sirala=indirim-yuzdesine-gore"


]




# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Taze sebzeler (patates hariç)"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()






















Scraping URL: https://www.migros.com.tr/sebze-c-66?sayfa=1&markalar=492&94=661,664,666,693,676,694,686,696,681,697,691,700,673,705,684,708,712,713,668,721,722,723,724,726,729,730,732,733,735,736,738,727,692,667,743,745,671,689,674,747,749,740,755&sirala=indirim-yuzdesine-gore
Product: Zencefil Kg, Price: 189.95 TRY
Product: Kuzu Kulağı Demet, Price: 26.95 TRY
Product: Soğan Arpacık Kg, Price: 84.95 TRY
Product: Aressa Soyulmuş Sarımsak 100 G, Price: 49.95 TRY
Product: Freshes Koçanda Tatlı Mısır 2 Adet Paket, Price: 99.95 TRY
Product: Pancar Kg, Price: 34.95 TRY
Product: Turp Kırmızı Demet, Price: 39.95 TRY
Product: Spesiyal Soğan File Kg, Price: 27.95 TRY
Product: Kereviz Kg, Price: 89.95 TRY
Product: Naturel Tatlı Kabak Kg (Hokkaido Kabak), Price: 42.95 TRY
Product: Kabak Tatlı (Butternut) Kg, Price: 32.95 TRY
Product: Alaşehir Asma Yaprağı (Ci) 455 G, Price: 119.95 TRY
Product: Fesleğen Demet, Price: 29.95 TRY
Product: Dereotu Adet, Price: 17.95 TRY
Product: Nane Adet, Price: 19.95 

In [326]:
patates=urunler_df[urunler_df['Ürün'].str.contains("patates", case=False)]
urunler_df=urunler_df[~urunler_df['Ürün'].str.contains("patates", case=False)]
data=veriekle("Taze sebzeler (patates hariç)",data,urunler_df)

In [327]:

import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data



# Define the list of pages to scrape from Migros
migros_pages  = ["https://www.migros.com.tr/sebze-c-66?sayfa=1&markalar=492&94=678&sirala=indirim-yuzdesine-gore"


]




# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)


all_data = migros_data 
product_df = pd.DataFrame(all_data)


driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Taze sebzeler (patates hariç)"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()


patates.index=len(patates)*["Patates"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=pd.concat([urunler_df,patates],axis=0)




















Scraping URL: https://www.migros.com.tr/sebze-c-66?sayfa=1&markalar=492&94=678&sirala=indirim-yuzdesine-gore
Product: Spesiyal Patates File Kg, Price: 27.95 TRY
Product: Patates Taze Mini Kg, Price: 49.95 TRY
Product: Patates Kumpirlik Kg, Price: 39.95 TRY
Product: Patates Yeni Mahsul Kg, Price: 11.5 TRY


In [328]:
data=veriekle("Patates",data,urunler_df)

In [329]:

import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    
    pages = ["https://www.carrefoursa.com/bakliyat/c/1152?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]



    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  = ["https://www.migros.com.tr/fasulye-c-2794?sayfa=1&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/mercimek-c-113c9?sayfa=1&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/nohut-c-2798?sayfa=1&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/nohut-c-2798?sayfa=1&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/barbunya-c-2791?sayfa=1&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/borulce-c-2792?sayfa=1&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/bakla-c-2790?sayfa=1&sirala=indirim-yuzdesine-gore"



]




# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Kuru baklagiller"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()























Scraping URL: https://www.migros.com.tr/fasulye-c-2794?sayfa=1&sirala=indirim-yuzdesine-gore
Product: Reis Kuru Fasulye 1Kg, Price: 149.95 TRY
Product: Hasata Derinkuyu Kuru Fasulyesi 1 Kg, Price: 124.95 TRY
Product: M Life Bio Organik Dermason Fasulye 1000 G, Price: 114.9 TRY
Product: M Life Maş Fasulyesi 500 G, Price: 34.9 TRY
Product: Doyum Organik Fasulye 1 Kg, Price: 128.95 TRY
Product: Yayla Gurme Maş Fasulye 500 G, Price: 44.95 TRY
Product: S.S Tire Organik Kuru Fasulye 1 Kg, Price: 124.95 TRY
Product: Reis Royal Maş Fasulye 500 G, Price: 82.95 TRY
Product: Reis Royal Siyah Fasulye 500 G, Price: 124.95 TRY
Product: Tat Dermason Fasulye 1 Kg, Price: 98.95 TRY
Product: Migros İri Dermason Fasulye 1000 G, Price: 64.95 TRY
Product: Reis Royal Kırmızı Fasulye 500G, Price: 140.95 TRY
Product: Yayla Dermason Fasulye 2 Kg, Price: 194.95 TRY
Product: Yayla Dermason Fasulye 1 Kg, Price: 99.95 TRY
Product: Migros Dermason Kuru Fasulye 1000 G, Price: 59.75 TRY
Product: Reis Royal Bombay Fas

In [330]:
data=veriekle("Kuru baklagiller",data,urunler_df)

In [331]:

import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    
    pages = ["https://www.carrefoursa.com/konserveler/c/1186?q=%3AbestSeller%3Acategory%3A1193%3AinStockFlag%3Atrue&text=#","https://www.carrefoursa.com/konserveler/c/1186?q=%3AbestSeller%3Acategory%3A1187%3AinStockFlag%3Atrue&text=#",
         "https://www.carrefoursa.com/salca-harc-ve-bulyon/c/1174?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All"]



    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  = ["https://www.migros.com.tr/arama?q=konserve&sayfa=1&kategori=1106&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=konserve&sayfa=2&kategori=1106&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=konserve&sayfa=3&kategori=1106&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=konserve&sayfa=4&kategori=1106&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=konserve&sayfa=5&kategori=1106&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=konserve&sayfa=6&kategori=1106&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=konserve&sayfa=7&kategori=1106&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=konserve&sayfa=8&kategori=1106&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/dondurulmus-patates-c-282c?sayfa=1&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/dondurulmus-sebze-c-282d?sayfa=1&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/dondurulmus-sebze-c-282d?sayfa=2&sirala=indirim-yuzdesine-gore"




]





# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})
urunler_df.index=len(urunler_df)*["Konserve edilmiş veya işlenmiş sebze içerikli ürünler (salça, turşu, zeytin vb. dahil)"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()
























Scraping URL: https://www.migros.com.tr/arama?q=konserve&sayfa=1&kategori=1106&sirala=indirim-yuzdesine-gore
Product: Tat Doğranmış Domates 400 G, Price: 29.95 TRY
Product: Ant Bahar Tahıllı Tohumlu Hindi Konserve 185 G, Price: 89.95 TRY
Product: Ant Bahar Hindi Tandır Konserve 2X120 G, Price: 151.95 TRY
Product: Ant Bahar Hindi Fileto Konserve 2X120 G, Price: 140.95 TRY
Product: Tamek Domates Salçası 830 G, Price: 57.95 TRY
Product: Kühne Acı Közlenmiş Biber 670 G, Price: 124.95 TRY
Product: Superfresh Ton Mix Mısırlı Ton 2 x 150 G, Price: 163.95 TRY
Product: Superfresh Ton Mix Tahıllı Ton 2x150 G, Price: 163.95 TRY
Product: Superfresh Ton Mix Fasulyeli Ton Balık 2 x 150 G, Price: 163.95 TRY
Product: Sera Acı Frenk Biber Turşusu 640 G, Price: 127.95 TRY
Product: Tukaş Domates Salçası 1650 G, Price: 114.95 TRY
Product: Sera Çağla Turşusu 700 G, Price: 74.95 TRY
Product: Tukaş Domates Salçası 830 G, Price: 52.95 TRY
Product: Dardanel Ton Klasik 3X75 G + Sardalya 125 G Bedava, Price: 214

In [332]:
data=veriekle("Konserve edilmiş veya işlenmiş sebze içerikli ürünler (salça, turşu, zeytin vb. dahil)",data,urunler_df)

In [333]:

import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    
    pages = ["https://www.carrefoursa.com/seker/c/1495?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]



    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data
# Define the list of pages to scrape from Migros
migros_pages  = ["https://www.migros.com.tr/arama?q=%C5%9Feker&sayfa=1&kategori=172&sirala=indirim-yuzdesine-gore"




]





# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Şeker"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()

























Scraping URL: https://www.migros.com.tr/arama?q=%C5%9Feker&sayfa=1&kategori=172&sirala=indirim-yuzdesine-gore
Product: Migros Pudra Şekeri 250 G, Price: 16.5 TRY
Product: Irmak Tekli Sargılı Küp Şeker 500 G (Poşet), Price: 33.95 TRY
Product: Coca-Cola Zero Sugar 250 Ml Cam Şişe, Price: 30.0 TRY
Product: Bor Şeker Küp Şeker 1 Kg, Price: 51.95 TRY
Product: Takita Kahverengi Toz Şeker 500 G, Price: 81.95 TRY
Product: Takita Kahverengi Küp Şeker 500 G, Price: 101.95 TRY
Product: Kayseri Şeker Kahverengi Şeker 500 G, Price: 59.95 TRY
Product: Coca-Cola Zero Sugar Kutu 6x250 ML, Price: 164.95 TRY
Product: Kayseri Şeker Kahverengi Küp Şeker 500 G, Price: 59.95 TRY
Product: Coca-Cola Zero Sugar Pet 1 L, Price: 30.95 TRY
Product: Nar Küp Şeker 1 Kg, Price: 41.95 TRY
Product: Dr.Oetker Pembe Pudra Şekeri 15 G, Price: 4.5 TRY
Product: Irmak Küp Şeker 750 G, Price: 33.95 TRY
Product: Coca-Cola Zero Sugar Fridgepack 8x200 ML, Price: 128.0 TRY
Product: Coca-Cola Zero Sugar Cam Şişe 6x250 ML, Price: 

In [334]:
data=veriekle("Şeker",data,urunler_df)

In [335]:


import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    
    pages = ["https://www.carrefoursa.com/kahvaltiliklar/c/1390?q=%3AbestSeller%3Acategory%3A1362%3AinStockFlag%3Atrue&show=All",
         "https://www.carrefoursa.com/kahvaltiliklar/c/1390?q=%3AbestSeller%3Acategory%3A1367%3AinStockFlag%3Atrue&show=All",
         "https://www.carrefoursa.com/kahvaltiliklar/c/1390?q=%3AbestSeller%3Acategory%3A1374%3AinStockFlag%3Atrue&text=#"]



    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  =  ["https://www.migros.com.tr/bal-c-420?sayfa=1&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/bal-c-420?sayfa=2&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/bal-c-420?sayfa=3&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/recel-marmelat-c-424?sayfa=1&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/recel-marmelat-c-424?sayfa=2&sirala=indirim-yuzdesine-gore"
         




]





# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Reçel, marmelat, bal vb. ürünler"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()


























Scraping URL: https://www.migros.com.tr/bal-c-420?sayfa=1&sirala=indirim-yuzdesine-gore
Product: Balparmak Yayla Çiçek Balı 225 G, Price: 154.9 TRY
Product: Balparmak Yayla Çiçek Balı 460 G, Price: 289.9 TRY
Product: Balparmak Süzme Çam Balı 460 G, Price: 289.9 TRY
Product: Balparmak Yüksek Yayla Çiçek Balı 460 G, Price: 344.9 TRY
Product: Beeo 190 G Propolis Arı Sütü Ham Bal Karışımı Çocuk, Price: 459.9 TRY
Product: Beeo Propolis Damla 20 Ml, Price: 349.9 TRY
Product: Beeo Çam Balı-Ham Bal 300 G, Price: 399.9 TRY
Product: Beeo Propolis Suda Çözünebilir Damla 20 Ml, Price: 349.9 TRY
Product: Beeo 190 G Propolis Arı Sütü Ham Bal Karışımı, Price: 499.9 TRY
Product: Beeo On The Go Propolis Karışım 7X7 G, Price: 124.9 TRY
Product: Beeo 180 G Propolis Fındık Kakao Ham Bal, Price: 274.9 TRY
Product: Beeo Polen 150 G, Price: 329.9 TRY
Product: Beeo Yayla Çiçek Balı- Ham Bal (Hakkari Yöresi) 300 G, Price: 399.9 TRY
Product: Balparmak Yayla ve Ova Çiçek Balı 850 G, Price: 377.9 TRY
Product: Bal

In [336]:
data=veriekle("Reçel, marmelat, bal vb. ürünler",data,urunler_df)

In [337]:

import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    
    pages = ["https://www.carrefoursa.com/cerez-cikolata-ve-biskuvi/c/1493?q=%3AbestSeller%3Acategory%3A1505%3AinStockFlag%3Atrue&show=All",
         "https://www.carrefoursa.com/cerez-cikolata-ve-biskuvi/c/1493?q=%3AbestSeller%3Acategory%3A1494%3AinStockFlag%3Atrue&show=All"]



    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  =  ["https://www.migros.com.tr/arama?q=%C3%A7ikolata&sayfa=1&kategori=70651&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=%C3%A7ikolata&sayfa=2&kategori=70651&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=%C3%A7ikolata&sayfa=3&kategori=70651&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=%C3%A7ikolata&sayfa=4&kategori=70651&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=%C3%A7ikolata&sayfa=5&kategori=70651&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=%C3%A7ikolata&sayfa=6&kategori=70651&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=%C3%A7ikolata&sayfa=7&kategori=70651&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=%C3%A7ikolata&sayfa=8&kategori=70651&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=%C3%A7ikolata&sayfa=9&kategori=70651&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=%C3%A7ikolata&sayfa=10&kategori=70651&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=%C3%A7ikolata&sayfa=11&kategori=70651&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=%C3%A7ikolata&sayfa=12&kategori=70651&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=%C3%A7ikolata&sayfa=13&kategori=70651&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=%C3%A7ikolata&sayfa=14&kategori=70651&sirala=indirim-yuzdesine-gore",
         




]




# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Çikolata ve şekerlemeler"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()



























Scraping URL: https://www.migros.com.tr/arama?q=%C3%A7ikolata&sayfa=1&kategori=70651&sirala=indirim-yuzdesine-gore
Product: Ülker Çikolatalı Gofret 36 G, Price: 9.5 TRY
Product: Kahve Dünyası Gofrik Antep Fıstıklı Sütlü 33 G, Price: 29.5 TRY
Product: Ülker Bütün Antep Fıstıklı Sütlü Kare Çikolata 65G, Price: 60.0 TRY
Product: Kinder Bueno Çikolata 43 gr, Price: 27.5 TRY
Product: Ülker Dido Trio Colors Çilekli 36.5 G, Price: 16.5 TRY
Product: Dido Trio Beyaz-Sütlü-Bitter Çikolatalı Gofret 36.5G, Price: 16.5 TRY
Product: Ülker Dido Gold Süt Reçeli Tadında Çikolatalı Gofret 36 G, Price: 16.5 TRY
Product: Ülker Dido Sütlü Çikolatalı Gofret 3Lü Paket 105G, Price: 35.5 TRY
Product: Ülker Laviva Dolgulu ve Bisküvili Bar 3x35 G, Price: 35.5 TRY
Product: Eti Browni Intense Çikolatalı Kek 50 g, Price: 18.5 TRY
Product: Eti Browni Intense Gold Çikolatalı Kek 48 g, Price: 18.5 TRY
Product: Nestlé Nesquik Sütlü Çikolatalı Gofret 26.7 G, Price: 10.0 TRY
Product: Eti Karam Gurme Bitter Çikolatalı Gof

In [338]:
data=veriekle("Çikolata ve şekerlemeler",data,urunler_df)

In [339]:


import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    
    pages = ["https://www.carrefoursa.com/multipack-dondurma/c/1270?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#",
         "https://www.carrefoursa.com/kap-dondurma/c/1261?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All",
         "https://www.carrefoursa.com/tek-dondurma/c/1266?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All"]



    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  =  ["https://www.migros.com.tr/arama?q=dondurma&sayfa=1&kategori=1051&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=dondurma&sayfa=2&kategori=1051&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=dondurma&sayfa=3&kategori=1051&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=dondurma&sayfa=4&kategori=1051&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=dondurma&sayfa=5&kategori=1051&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=dondurma&sayfa=6&kategori=1051&sirala=indirim-yuzdesine-gore",
         




]





# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Dondurma"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()





























Scraping URL: https://www.migros.com.tr/arama?q=dondurma&sayfa=1&kategori=1051&sirala=indirim-yuzdesine-gore
Product: Arzen Prebiyotik Sade Çikolatalı Dondurma 500 Ml, Price: 133.16 TRY
Product: Maraşım Türk Kahveli Sade 500 Ml, Price: 142.9 TRY
Product: Panda Efsane 1985 Stix Dondurma 6'lı 420 Ml, Price: 169.9 TRY
Product: Golf Maraşım Çikolata - Sade 500 Ml, Price: 137.9 TRY
Product: Golf Maraşım Antep Fıstığı - Sade 500 Ml, Price: 137.9 TRY
Product: Golf Maraşım Sade 500 Ml, Price: 137.9 TRY
Product: Magnum Mini Double Euphorıa Limon Ahududu Badem 353 ML, Price: 169.9 TRY
Product: Magnum Mini Classic, Badem, Beyaz 345 Ml 6'lı, Price: 169.9 TRY
Product: Magnum Mini Badem 6'lı 345 Ml, Price: 169.9 TRY
Product: Magnum Mini Cookie & Badem 345 Ml, Price: 169.9 TRY
Product: Magnum Mini Classic 345 Ml 6'lı, Price: 169.9 TRY
Product: Magnum Mini Double Çikolata, Karadut & Böğürtlen, Karamel (6'lı), Price: 169.9 TRY
Product: Carte d'Or Vişne Vanilya Bitter Çikolata 850 ml, Price: 139.9 TRY
P

In [340]:
data=veriekle("Dondurma",data,urunler_df)

In [341]:


import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    
    pages = ["https://www.carrefoursa.com/seker-tuz-ve-baharat/c/1159?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All",
         "https://www.carrefoursa.com/ketcap-mayonez-ve-soslar/c/1209?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All"]



    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  =  ["https://www.migros.com.tr/sos-c-113fc?sayfa=1&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/sos-c-113fc?sayfa=2&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/sos-c-113fc?sayfa=3&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/sos-c-113fc?sayfa=4&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/sos-c-113fc?sayfa=5&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/sos-c-113fc?sayfa=6&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/tuz-baharat-harc-c-77?sayfa=1&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/tuz-baharat-harc-c-77?sayfa=2&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/tuz-baharat-harc-c-77?sayfa=3&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/tuz-baharat-harc-c-77?sayfa=4&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/tuz-baharat-harc-c-77?sayfa=5&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/tuz-baharat-harc-c-77?sayfa=6&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/tuz-baharat-harc-c-77?sayfa=7&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/tuz-baharat-harc-c-77?sayfa=8&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/tuz-baharat-harc-c-77?sayfa=9&sirala=indirim-yuzdesine-gore",

         




]





# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Başka yerde sınıflandırılamayan diğer gıda ürünleri (tuz, kabartma maddeleri, sirke, ketçap, mayonez vb.)"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()































Scraping URL: https://www.migros.com.tr/sos-c-113fc?sayfa=1&sirala=indirim-yuzdesine-gore
Product: Tukaş Ketçap 400 G, Price: 31.95 TRY
Product: Tat Sarımsaklı Mayonez 205 G, Price: 56.95 TRY
Product: Tat Ranch Sos 220 G, Price: 54.95 TRY
Product: Tariş Üzüm Sirkesi 1 L, Price: 89.95 TRY
Product: Burcu Hardal 320 G, Price: 59.95 TRY
Product: Calve Mayoçap 405 G, Price: 98.95 TRY
Product: Maille Dijon Hardal 200 G, Price: 252.95 TRY
Product: Tukaş Basilico Makarna Sosu 360 G, Price: 37.95 TRY
Product: Kühne Soya Sosu 250 Ml, Price: 104.95 TRY
Product: Kühne Üzüm Sirkesi 500 Ml Pet, Price: 49.95 TRY
Product: Kühne Elma Sirkesi Cam 500 Ml, Price: 69.95 TRY
Product: Kühne Üzüm Sirkesi 1000 Ml Pet, Price: 82.95 TRY
Product: Kühne Balsamik Sirke 250 Ml, Price: 119.95 TRY
Product: Kühne Üzüm Sirkesi Cam 500 Ml, Price: 58.95 TRY
Product: Kühne Beyaz Sirke 1000 Ml, Price: 39.95 TRY
Product: Kühne Balsamik Sirke 500 Ml, Price: 167.95 TRY
Product: Kühne Elma Sirkesi 500 Ml Pet, Price: 59.95 TRY
P

In [342]:
data=veriekle("Başka yerde sınıflandırılamayan diğer gıda ürünleri (tuz, kabartma maddeleri, sirke, ketçap, mayonez vb.)",data,urunler_df)

In [343]:

import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    
    pages = ["https://www.carrefoursa.com/kahve/c/1467?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All"]



    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  =   ["https://www.migros.com.tr/arama?q=kahve&sayfa=1&kategori=1142&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=kahve&sayfa=2&kategori=1142&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=kahve&sayfa=3&kategori=1142&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=kahve&sayfa=4&kategori=1142&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=kahve&sayfa=5&kategori=1142&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=kahve&sayfa=6&kategori=1142&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=kahve&sayfa=7&kategori=1142&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=kahve&sayfa=8&kategori=1142&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=kahve&sayfa=9&kategori=1142&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=kahve&sayfa=10&kategori=1142&sirala=indirim-yuzdesine-gore",

         




]






# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Kahve"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()

































Scraping URL: https://www.migros.com.tr/arama?q=kahve&sayfa=1&kategori=1142&sirala=indirim-yuzdesine-gore
Product: Tchibo Gold Selection Çözünebilir Kahve 50 G, Price: 144.95 TRY
Product: Tchibo Gold Selection Çözünebilir Kahve Ekopaket 150 G, Price: 219.95 TRY
Product: Tchibo Gold Selection Kavanoz 100 G, Price: 229.95 TRY
Product: Tchibo Gold Selection Çözünebilir Kahve 75 G Ekopaket, Price: 114.95 TRY
Product: Starbucks White Mocha Kahve Karışımı 24 G, Price: 19.95 TRY
Product: Starbucks Caramel Latte Premium Kahve Karışımı 21,5g, Price: 19.95 TRY
Product: Starbucks Vanilla Latte Premium Kahve Karışımı 21.5 G, Price: 19.95 TRY
Product: Starbucks Cappuccino Premium Kahve Karışımı 18 G, Price: 19.95 TRY
Product: Starbucks Caffe Mocha Premium Kahve Karışımı 22 G, Price: 19.95 TRY
Product: Nescafe Gold Filtre Kahve 250 G, Price: 199.95 TRY
Product: Nescafe Gold Filtre Kahve 500 G, Price: 389.95 TRY
Product: Nescafe Gold Espresso Ekonomik Paket 80 G, Price: 165.0 TRY
Product: Nescafe Gol

In [344]:
data=veriekle("Kahve",data,urunler_df)

In [345]:

import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    
    pages = ["https://www.carrefoursa.com/cay-/c/1455?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All"]



    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  =  ["https://www.migros.com.tr/arama?q=%C3%A7ay&sayfa=1&kategori=1141&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=%C3%A7ay&sayfa=2&kategori=1141&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=%C3%A7ay&sayfa=3&kategori=1141&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=%C3%A7ay&sayfa=4&kategori=1141&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=%C3%A7ay&sayfa=5&kategori=1141&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=%C3%A7ay&sayfa=6&kategori=1141&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=%C3%A7ay&sayfa=7&kategori=1141&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=%C3%A7ay&sayfa=8&kategori=1141&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=%C3%A7ay&sayfa=9&kategori=1141&sirala=indirim-yuzdesine-gore",


         




]







# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Çay ve bitki çayları"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()


































Scraping URL: https://www.migros.com.tr/arama?q=%C3%A7ay&sayfa=1&kategori=1141&sirala=indirim-yuzdesine-gore
Product: Teekanne Ingwer Zencefilli Limonlu Bitki Çayı 20'li 35g, Price: 69.9 TRY
Product: Teekanne Thai Ingwer Mango Arm Zencefil Çay 20'li 45g, Price: 69.9 TRY
Product: Teekanne Minze Nane Çayı 20'li 45g, Price: 79.9 TRY
Product: Herby Ice Tea Açai Ananas 250 Ml, Price: 59.95 TRY
Product: Herby Ice Tea Berry Hibiscus 250 Ml, Price: 59.95 TRY
Product: Herby Ice Tea Cool Lime 250 Ml, Price: 59.95 TRY
Product: Lipton Earl Grey Bardak Poşet Çay 100'lü 200 G, Price: 240.0 TRY
Product: Lipton Bardak Poşet Çay Yellow Label 100, Price: 240.0 TRY
Product: Lipton Doğu Karadeniz Çayı 100'lü Bardak Poşet 200 G, Price: 190.0 TRY
Product: Lipton Demlik Poşet Çay Yellow Label 100'Lü, Price: 180.0 TRY
Product: Ahmad Tea English Breakfast Siyah Dökme Çay 200 G, Price: 249.5 TRY
Product: Ahmad Tea English Breakfast Dökme Çay 400 G, Price: 449.5 TRY
Product: Lipton Demlik Poşet Çay Doğu Karadeni

In [346]:
data=veriekle("Çay ve bitki çayları",data,urunler_df)

In [347]:

import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    
    pages = ["https://www.carrefoursa.com/search/?q=kakao%3AbestSeller%3AinStockFlag%3Atrue&text=kakao#"]



    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  =  ["https://www.migros.com.tr/kakao-c-2894?sayfa=1&sirala=indirim-yuzdesine-gore"]







# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Toz kakao"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()





































Scraping URL: https://www.migros.com.tr/kakao-c-2894?sayfa=1&sirala=indirim-yuzdesine-gore
Product: Dr.Oetker Glutensiz Kakao 70 G, Price: 52.95 TRY
Product: Pakmaya Kakao 2'li 2x50 G, Price: 58.95 TRY
Product: Pakmaya Kakao 2'li 2X25 G, Price: 30.45 TRY
Product: Ülker Poşet Toz Kakao 50 G, Price: 27.95 TRY
Product: Dr.Oetker Kakao 2X25 G, Price: 32.45 TRY
Product: Dr.Oetker Kakao 100 G, Price: 62.95 TRY
Product 1: Carrefour Kakao 100 g, Price: 34.75 TL
Product 2: Dr Oetker Kakao 50 G, Price: 32.5 TL
Product 3: Ülker Kakao 50 gr, Price: 28.9 TL
Product 4: Dr Oetker Kakao 100 Gr, Price: 62.9 TL
Product 5: Kakao kg, Price: 599.9 TL
Product 6: Pakmaya Kakao 2x25 Gr, Price: 30.5 TL
Product 7: Nestle Toz Kakao 100 Gr, Price: 54.9 TL
Product 8: Kent Boringer Kakao 100 Gr, Price: 41.52 TL
Product 9: Dr Oetker Gourmet Dark Kakao 50 G, Price: 34.5 TL
Product 10: Kireçburnu Glutensiz KakaoFındıklı Karabuğdaylı Kurabiye 180 G, Price: 81.55 TL


In [348]:
data=veriekle("Toz kakao",data,urunler_df)

In [349]:

import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    
    pages = ["https://www.carrefoursa.com/sular/c/1411?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All"]



    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data
# Define the list of pages to scrape from Migros
migros_pages  =  ["https://www.migros.com.tr/arama?q=su&sayfa=1&kategori=6&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=su&sayfa=2&kategori=6&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/arama?q=su&sayfa=3&kategori=6&sirala=indirim-yuzdesine-gore"


         




]







# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Su ve maden suyu"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()








































Scraping URL: https://www.migros.com.tr/arama?q=su&sayfa=1&kategori=6&sirala=indirim-yuzdesine-gore
Product: Hayat Su 0,75 L, Price: 17.5 TRY
Product: Saka Doğal Mineralli Su 6x1 L, Price: 88.5 TRY
Product: Pınar Yaşam Pınarım 5 L, Price: 41.95 TRY
Product: Erikli Doğal Kaynak Suyu 1L Pet Şişe, Price: 23.95 TRY
Product: Mila Cam Şişe Su 330 Ml, Price: 19.95 TRY
Product: Saol Vitamin Water Antioxidant Aromlı İçecek 500 Ml, Price: 41.95 TRY
Product: Perrier - Doğal Mineralli Su - 6 X 200 Ml, Price: 375.95 TRY
Product: Erikli Doğal Kaynak Suyu Cam 750 ML, Price: 72.5 TRY
Product: Mila Doğal Kaynak Suyu 1.5 L, Price: 12.95 TRY
Product: Buzdağı Doğal Mineralli Su 0,5 L, Price: 8.5 TRY
Product: Saol Vitamin Water Multi-V 500 Ml, Price: 41.95 TRY
Product: San Pellegrino Doğal Mineralli Su 750 Ml, Price: 137.95 TRY
Product: Pürsu Elite Doğal Kaynak Suyu Cam Şişe 750 Ml, Price: 36.25 TRY
Product: Pürsu Elite Doğal Kaynak Suyu Cam Şişe 330 Ml, Price: 21.5 TRY
Product: Buzdağı Doğal Mineralli Su 

In [350]:
data=veriekle("Su ve maden suyu",data,urunler_df)

In [351]:

import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    
    pages = ["https://www.carrefoursa.com/gazli-icecekler/c/1418?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All",
         "https://www.carrefoursa.com/ayran-kefir/c/1485?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All",
         "https://www.carrefoursa.com/soguk-cay/c/1450?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All",
         "https://www.carrefoursa.com/salgam-suyu/c/1488?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]



    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  = ["https://www.migros.com.tr/gazli-icecek-c-80?sayfa=1&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/gazli-icecek-c-80?sayfa=2&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/gazli-icecek-c-80?sayfa=3&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/gazli-icecek-c-80?sayfa=4&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/gazli-icecek-c-80?sayfa=5&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/gazli-icecek-c-80?sayfa=6&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/salgam-suyu-c-46e?sayfa=1&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/aroma-suruplari-c-472?sayfa=1&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/ayran-c-47a?sayfa=1&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/kefir-c-47b?sayfa=1&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/kefir-c-47b?sayfa=2&sirala=indirim-yuzdesine-gore"


         




]







# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Alkolsüz içecekler (meşrubat, ayran vb.)"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()











































Scraping URL: https://www.migros.com.tr/gazli-icecek-c-80?sayfa=1&sirala=indirim-yuzdesine-gore
Product: Pepsi Raspberry Ahududu Aromalı Pet Kola 1 L, Price: 35.0 TRY
Product: Black Bruin 1 Litre, Price: 40.0 TRY
Product: Mountain Dew Turunçgil Aromalı Gazoz Kutu 330 ml, Price: 30.0 TRY
Product: Mountain Dew Turunçgil Aromalı Gazoz Pet 450 ml, Price: 30.0 TRY
Product: Uludağ Efsane Şekersiz 6X250 Ml Cam, Price: 130.95 TRY
Product: Uludağ Efsane Gazoz 6X250 Ml, Price: 130.95 TRY
Product: Pepsi Kola Cam 6x200 ml, Price: 106.5 TRY
Product: Fruko Gazoz Pet 1,5 L, Price: 35.0 TRY
Product: Schweppes Mandalina Cam Şişe 6x250 ML, Price: 223.0 TRY
Product: Çamlıca Frambuaz Aromalı Gazoz Cam Şişe 6x200 Ml, Price: 64.0 TRY
Product: Fanta Portakal 500 Ml, Price: 30.0 TRY
Product: Uludağ Efsane Gazoz 2,5 L, Price: 52.95 TRY
Product: Yedigün Portakal 4x1 L, Price: 133.0 TRY
Product: Schweppes Pink Greyfurt Cam 250 Ml, Price: 42.0 TRY
Product: Uludağ Efsane Portakal Pet 1 L, Price: 30.5 TRY
Product: 

In [352]:
data=veriekle("Alkolsüz içecekler (meşrubat, ayran vb.)",data,urunler_df)

In [353]:



import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = False  # Tarayıcı görünür modda çalışacak

# WebDriver'ı başlat
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to clean and convert price text to float
def clean_price(price_text):
    # Remove any non-numeric characters except for commas and dots
    price_text = re.sub(r'[^\d,.]', '', price_text)
    # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
    price_text = price_text.replace(',', '.')
    try:
        return float(price_text)
    except ValueError:
        return None

# Function to scrape product names and prices from Migros for multiple URLs
def scrape_migros_products(pages):
    product_data = []
    
    for page in pages:
        print(f"Scraping URL: {page}")
        driver.get(page)

        try:
            # Wait for the product names to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.product-name'))
            )

            # Get product names and prices
            products = driver.find_elements(By.CSS_SELECTOR, '.product-name')
            prices = driver.find_elements(By.CSS_SELECTOR, '.price-new')

            # Ensure we have matching names and prices
            if len(products) != len(prices):
                print(f"Warning: Number of products and prices do not match on page {page}")

            for i in range(len(products)):
                try:
                    product_name = products[i].text.strip()
                    product_price_text = prices[i].text.strip()
                    product_price = clean_price(product_price_text)  # Convert to float
                    product_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                    print(f"Product: {product_name}, Price: {product_price} TRY")
                except Exception as e:
                    print(f"Error scraping product {i} on page {page}: {e}")

        except Exception as e:
            print(f"Error loading products on page {page}: {e}")

        # Sleep to avoid being detected as a bot
        sleep(2)
    
    return product_data

# Function to scrape product names and prices from Carrefour for multiple URLs
def scrape_carrefour_products():
    
    pages = ["https://www.carrefoursa.com/meyve-suyu/c/1430?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All"]



    
    carrefour_data = []
    
    for url in pages:
    
            # Send a GET request to fetch the HTML content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all product cards
            product_cards = soup.find_all("div", class_="productCardData")
            
            # Loop through each product card and extract the product name and price
            for index, product_card in enumerate(product_cards):
                # Extract product name
                product_name = product_card.find("input", {"id": "productNamePost"})['value']
                
                # Extract product price (from the input tag with id="productPricePost")
                product_price = product_card.find("input", {"id": "productPricePost"})['value']
                
                # Convert price to float
                try:
                    product_price = float(product_price)
                except ValueError:
                    product_price = None  # Handle cases where the price might not be convertible
                
                # Print product name and price
                print(f"Product {index + 1}: {product_name}, Price: {product_price} TL")
                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})

        else:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return carrefour_data

# Define the list of pages to scrape from Migros
migros_pages  = ["https://www.migros.com.tr/meyve-suyu-c-46c?sayfa=1&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/meyve-suyu-c-46c?sayfa=2&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/meyve-suyu-c-46c?sayfa=3&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/meyve-suyu-c-46c?sayfa=4&sirala=indirim-yuzdesine-gore",
         "https://www.migros.com.tr/meyve-suyu-c-46c?sayfa=5&sirala=indirim-yuzdesine-gore",



         




]








# Scrape products from Migros and Carrefour
migros_data = scrape_migros_products(migros_pages)
carrefour_data = scrape_carrefour_products()

# Combine both datasets into one DataFrame
all_data = migros_data + carrefour_data
product_df = pd.DataFrame(all_data)

# Close the browser
driver.quit()





urunler_df = product_df.copy()
urunler_df.columns=["Ürün",str(bugün)]
urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

urunler_df.index=len(urunler_df)*["Meyve ve sebze suları"]
urunler_df=urunler_df.drop_duplicates()
urunler_df=urunler_df.dropna()














































Scraping URL: https://www.migros.com.tr/meyve-suyu-c-46c?sayfa=1&sirala=indirim-yuzdesine-gore
Product: Torku Detoks Sirkeli Üzümlü İçecek 250 Ml, Price: 33.5 TRY
Product: Cappy Şeftali Nektarı Karton Kutu 1 L, Price: 37.5 TRY
Product: Cappy Atom Ballı Karışık Meyve Nektarı Karton Kutu 1 L, Price: 37.5 TRY
Product: Tamek Vişne Nektarı 1L, Price: 47.95 TRY
Product: Cappy Karışık Meyve Nektarı Karton Kutu 1 L, Price: 37.5 TRY
Product: Tamek Şeftali Nektarı 1 L, Price: 40.95 TRY
Product: Tamek Karışık Meyve Nektarı 1 Lt, Price: 40.95 TRY
Product: Tamek Kayısı Nektarı 1 Lt, Price: 40.95 TRY
Product: Dimes Mango Aromalı İçecek 1 L, Price: 44.95 TRY
Product: Dimes Ananaslı İçecek 1 L, Price: 44.95 TRY
Product: Dimes Ekşi Elma 1 L, Price: 44.95 TRY
Product: Dimes Karpuz Ve Çilekli İçecek 1 L, Price: 44.95 TRY
Product: Dimes 7/24 Enerji Karışık Meyveli İçecek 1 L, Price: 44.95 TRY
Product: Cappy %100 Elma Suyu Karton Kutu 1 L, Price: 45.0 TRY
Product: Cappy %100 Karışık Meyve Suyu Karton Kutu 

In [354]:
data=veriekle("Meyve ve sebze suları",data,urunler_df)

In [355]:
data=data.drop_duplicates()

In [400]:
# Function to convert numeric columns to float and drop rows where conversion fails
def convert_to_float_and_drop_non_numeric(df):
    numeric_columns = df.columns[1:]  # Exclude the 'Ürün' column
    # Attempt to convert all numeric columns to float
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Drop rows where all numeric columns have NaN (i.e., non-convertible rows)
    df_cleaned = df.dropna(subset=numeric_columns, how='all')
    return df_cleaned

# Apply the conversion and cleaning process
df_cleaned = convert_to_float_and_drop_non_numeric(data.copy())



In [405]:
# Function to fill NaN values from both right to left and left to right
def fill_nan_both_directions(row):

    filled_row = row[::-1].fillna(method='ffill')[::-1]

    filled_row = filled_row.fillna(method='ffill')
    return filled_row

def fill_nan_both_directions_corrected(df):
    numeric_columns = df.columns[1:]  # Exclude the 'Ürün' column
    df[numeric_columns] = df[numeric_columns].apply(fill_nan_both_directions, axis=1)
    return df


df_filled_corrected = fill_nan_both_directions_corrected(df_cleaned)

C:\Users\Bora\AppData\Local\Temp\ipykernel_32600\2609800991.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  filled_row = row[::-1].fillna(method='ffill')[::-1]
C:\Users\Bora\AppData\Local\Temp\ipykernel_32600\2609800991.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  filled_row = filled_row.fillna(method='ffill')


In [408]:
ağırlıklar=pd.read_csv("C:/Users/Bora/Documents/GitHub/streamlit-inflation/ağırlıklar.csv")
ağırlıklar=ağırlıklar.set_index(ağırlıklar["Ürün"])
ağırlıklar=ağırlıklar.drop("Ürün",axis=1)
ağırlıklar


,Ağırlık,2024-10-07,Endeks_2024-10-06,Endeks_2024-10-07,Ağırlıklı Endeks_2024-10-07,2024-10-08,Endeks_2024-10-08,Ağırlıklı Endeks_2024-10-08,2024-10-09,Endeks_2024-10-09,Ağırlıklı Endeks_2024-10-09
Ürün,,,,,,,,,,,
"Alkolsüz içecekler (meşrubat, ayran vb.)",1.372423,-0.073669,100,99.926331,1.371412,-0.024533,99.901817,1.371076,0.074137,99.975881,1.372092
"Başka yerde sınıflandırılamayan diğer gıda ürünleri (tuz, kabartma maddeleri, sirke, ketçap, mayonez vb.)",1.480075,-0.289995,100,99.710005,1.475783,-0.019029,99.691031,1.475502,-0.130966,99.560469,1.473569
Dana eti,9.119303,-1.044107,100,98.955893,9.024088,0.000000,98.955893,9.024088,0.000000,98.955893,9.024088
Diğer etler ve yenilebilir sakatatlar,0.309776,0.000000,100,100.000000,0.309776,0.000000,100.000000,0.309776,0.000000,100.000000,0.309776
"Diğer fırıncılık ürünleri (bisküvi, kek, kraker, yufka, baklava vb.)",4.730758,0.000000,100,100.000000,4.730758,0.000000,100.000000,4.730758,0.000000,100.000000,4.730758
"Diğer süt ürünleri (yoğurt, hazır sütlü tatlı vb.)",2.177561,-0.080063,100,99.919937,2.175817,0.000000,99.919937,2.175817,-0.138134,99.781914,2.172812
Dondurma,0.471012,0.000000,100,100.000000,0.471012,-0.352105,99.647895,0.469353,0.000000,99.647895,0.469353
Ekmek,8.776321,0.000000,100,100.000000,8.776321,0.000000,100.000000,8.776321,0.000000,100.000000,8.776321
Kahvaltılık tahıl ürünleri,0.151911,0.711814,100,100.711814,0.152992,-0.553369,100.154505,0.152145,0.000000,100.154505,0.152145


In [411]:
data1=df_filled_corrected.copy()

In [409]:
gfe=pd.read_csv("C:/Users/Bora/Documents/GitHub/streamlit-inflation/gfe.csv")
gfe=gfe.set_index(pd.to_datetime(gfe["Tarih"]))
gfe=gfe.drop("Tarih",axis=1)
gfe

,Endeks
Tarih,
2024-10-06,100.000000
2024-10-07,99.828328
2024-10-08,99.856812
2024-10-09,100.511747


In [415]:
degisim=(((data1.iloc[:,-1]/data1.iloc[:,-2])-1)*100).fillna(0).groupby(level=0).mean().sort_index()


In [416]:
ağırlıklar[f"{bugün}"]=degisim.values

In [417]:
ağırlıklar[f"Endeks_{bugün}"]=ağırlıklar[f"Endeks_{dün}"]*(1+(ağırlıklar[bugün]/100))

In [418]:
ağırlıklar[f"Ağırlıklı Endeks_{bugün}"]=ağırlıklar[f"Endeks_{bugün}"]*ağırlıklar["Ağırlık"]/100

In [419]:
gfe.loc[pd.to_datetime(bugün)]=ağırlıklar[f"Ağırlıklı Endeks_{bugün}"].sum()
gfe.to_csv("C:/Users/Bora/Documents/GitHub/streamlit-inflation/gfe.csv",index=True)

In [420]:
endeks_sutunlari = ağırlıklar.filter(like='Endeks_')
endeksler = [col for col in ağırlıklar.columns if col.startswith('Endeks_')]
ağırlıklar[endeksler].to_csv("C:/Users/Bora/Documents/GitHub/streamlit-inflation/endeksler.csv",index=True)

In [421]:
ağırlıklar.to_csv("C:/Users/Bora/Documents/GitHub/streamlit-inflation/ağırlıklar.csv",index=True)

In [474]:
data1.to_csv("sepet.csv",index=True)

In [482]:
data1.to_csv("C:/Users/Bora/Documents/GitHub/streamlit-inflation/sepet.csv",index=True)

In [433]:
data1["Değişim"]=((data1.iloc[:,-1].values/data1.iloc[:,1].values)-1)*100

In [483]:
gfe

,Endeks
Tarih,
2024-10-06,100.000000
2024-10-07,99.828328
2024-10-08,99.856812
2024-10-09,100.520503
